# Bivariate portfolio sorts
We extend the univariate portfolio analysis of **Univariate Portfolio Sorts** to **Bivariate sorts**, which means we assign stocks to portfolios based on two characteristics. Bivariate sorts are regularly used in the academic asset pricing literature and are the basis for factors in the Fama-French three-factor model. We form portfolios on firm size and the book-to-market ratio. To calculate book-to-market ratios, accounting data is required, which necessitates additional steps during portfolio formation.
In the end, we demonstrate how to form portfolios on two sorting variables using so-called independent and dependent portfolio sorts.

In [1]:
#Install the necessary packages
!pip install pandas 
!pip install pandas_datareader
!pip install python-dotenv
!pip install psycopg2-binary

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
#Import the necessary modules

from sqlalchemy import create_engine
from plotnine import *
from regtabletotext import prettify_result
from datetime import datetime
import numpy as np
import sqlite3
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Data preparation
First, we load the necessary data from our SQLite database introduced in Accessing and Managing Financial Data and WRDS, CRSP, and Compustat. We conduct portfolio sorts based on the CRSP sample. 

***!Important! Make sure to download from BlackBoard the file FamaFrench.sqlite and store it in your local drive ***

In [3]:
your_path = input("Please specify the local path where you stored the db FamaFrench.sqlite: ") 
database = sqlite3.connect(database= f"{your_path}\FamaFrench5000.sqlite") 

crsp_monthly = (pd.read_sql_query(
    sql=("SELECT permno, gvkey, month, ret_excess, mktcap," 
         "mktcap_lag, exchange FROM crsp_monthly"),
    con=database,
    parse_dates={"month"})
  .dropna()
)

Please specify the local path where you stored the db FamaFrench.sqlite: C:\Users\rober\Downloads


Further, we utilize accounting data. We only need book equity data in this application, which we select from our database. Additionally, we convert the variable datadate to its monthly value, as we only consider monthly returns here and do not need to account for the exact date.

In [4]:
book_equity = (pd.read_sql_query(
    sql="SELECT gvkey, datadate, be FROM compustat",
    con=database, 
    parse_dates={"datadate"})
  .dropna()
  .assign(
    month=lambda x: (
      pd.to_datetime(x["datadate"]).dt.to_period("M").dt.to_timestamp()
    )
  )
)

## Book-to-Market Ratio
A fundamental problem in handling accounting data is the *look-ahead bias*; we must not include data in forming a portfolio that was not available knowledge at the time. Of course, researchers have more information when looking into the past than agents actually had at that moment. However, abnormal excess returns from a trading strategy should not rely on an information advantage because the differential cannot be the result of informed agents’ trades. Hence, we have to lag accounting information.

We continue to lag market capitalization and firm size by one month. Then, we compute the book-to-market ratio, which relates a firm’s book equity to its market equity. Firms with high (low) book-to-market ratio are called value (growth) firms. After matching the accounting and market equity information from the same month, we lag book-to-market by six months. This is a sufficiently conservative approach because accounting information is usually released well before six months pass.

Having both variables, i.e., firm size lagged by one month and book-to-market lagged by six months, we merge these sorting variables to our returns using the *sorting_date*-column created for this purpose. The final step in our data preparation deals with differences in the frequency of our variables. Returns and firm size are recorded monthly. Yet, the accounting information is only released on an annual basis. Hence, we only match book-to-market to one month per year and have eleven empty observations. To solve this frequency issue, we carry the latest book-to-market ratio of each firm to the subsequent months, i.e., we fill the missing observations with the most current report. This is done via the *fillna(method="ffill")-function* after sorting by date and firm (which we identify by permno and gvkey) and on a firm basis (which we do by .groupby() as usual). We filter out all observations with accounting data that is older than a year. As the last step, we remove all rows with missing entries because the returns cannot be matched to any annual report.

In [5]:
me = (crsp_monthly
  .assign(sorting_date=lambda x: x["month"]+pd.DateOffset(months=1))
  .rename(columns={"mktcap": "me"})
  .get(["permno", "sorting_date", "me"])
)

bm = (book_equity
  .merge(crsp_monthly, how="inner", on=["gvkey", "month"])
  .assign(bm=lambda x: x["be"]/x["mktcap"],
          sorting_date=lambda x: x["month"]+pd.DateOffset(months=6))
  .assign(comp_date=lambda x: x["sorting_date"])
  .get(["permno", "gvkey", "sorting_date", "comp_date", "bm"])
)

data_for_sorts = (crsp_monthly
  .merge(bm, 
         how="left", 
         left_on=["permno", "gvkey", "month"], 
         right_on=["permno", "gvkey", "sorting_date"])
  .merge(me, 
         how="left", 
         left_on=["permno", "month"], 
         right_on=["permno", "sorting_date"])
  .get(["permno", "gvkey", "month", "ret_excess", 
        "mktcap_lag", "me", "bm", "exchange", "comp_date"])
)

data_for_sorts = (data_for_sorts
  .sort_values(by=["permno", "gvkey", "month"])
  .groupby(["permno", "gvkey"])
  .apply(lambda x: x.assign(
      bm=x["bm"].fillna(method="ffill"), 
      comp_date=x["comp_date"].fillna(method="ffill")
    )
  )
  .reset_index(drop=True)
  .assign(threshold_date = lambda x: (x["month"]-pd.DateOffset(months=12)))
  .query("comp_date > threshold_date")
  .drop(columns=["comp_date", "threshold_date"])
  .dropna()
)

C:\Users\rober\AppData\Local\Temp/ipykernel_73032/4024672676.py:28: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


The last step of preparation for the portfolio sorts is the computation of breakpoints. We continue to use the same function of univariate portfolio sorting, allowing for the specification of exchanges to be used for the breakpoints. Additionally, we reintroduce the argument *sorting_variable* into the function for defining different sorting variables.

In [6]:
def assign_portfolio(data, exchanges, sorting_variable, n_portfolios):
    """Assign portfolio for a given sorting variable."""
    
    breakpoints = (data
      .query(f"exchange in {exchanges}")
      .get(sorting_variable)
      .quantile(np.linspace(0, 1, num=n_portfolios+1), 
                interpolation="linear")
      .drop_duplicates()
    )
    breakpoints.iloc[0] = -np.Inf
    breakpoints.iloc[breakpoints.size-1] = np.Inf
    
    assigned_portfolios = pd.cut(
      data[sorting_variable],
      bins=breakpoints,
      labels=range(1, breakpoints.size),
      include_lowest=True,
      right=False
    )
    
    return assigned_portfolios

After these data preparation steps, we present bivariate portfolio sorts on an independent and dependent basis.